In [1]:
import tweepy
import pandas as pd
import datetime as datetime

auth = tweepy.OAuth2BearerHandler("AAAAAAAAAAAAAAAAAAAAAH6nYwEAAAAAK%2BezpbKj7NUWoAMW6SbFOfFruD4%3Dj134xtdV6I1kcGl4a1wnAaa7N2W0hQCA9avu7sDyvEEVWjIwmy")
api = tweepy.API(auth)

In [2]:
#Searching the archive for tweets mentioning the mavs

mavsSearch = api.search_full_archive('dev', 'mavs', maxResults = 100)

TooManyRequests: 429 Too Many Requests
Request exceeds account’s current package request limits. Please upgrade your package and retry or contact Twitter about enterprise access.

In [8]:
#Viewing the first tweet

import json
status = mavsSearch[0]
json_str = json.dumps(status._json)
parsed = json.loads(json_str)

print(json.dumps(parsed, indent=4, sort_keys=True))

NameError: name 'mavsSearch' is not defined

In [9]:
#Creates the mineTweet function which extracts the information from a query and returns a dictionary with tweet data
#BUG: hashtags label also collects other random data from entities

def mineTweets(query, fromDate=datetime.datetime.now()):
        data = []
        fromDate = datetime.datetime.strptime(str(fromDate), "%Y-%m-%d %H:%M:%S.%f").strftime('%Y%m%d%H%M')

        searchResult = api.search_full_archive('dev', query, fromDate=fromDate, maxResults = 100)
        for item in searchResult:
                mined = {
                        'tweet_id':        item.id,
                        'name':            item.user.name,
                        'screen_name':     item.user.screen_name,
                        'retweet_count':   item.retweet_count,
                        'text':            item.text,
                        'hashtags':        item.entities['hashtags'],
                        'location':        item.place,
                        'is_verified':     item.user.verified,
                        'created_at':      item.created_at
                }
                try:
                        mined['retweet_text'] = item.retweeted_status.full_text
                except:
                        mined['retweet_text'] = 'None'
                try:
                        mined['quote_text'] = item.quoted_status.full_text
                        mined['quote_screen_name'] = status.quoted_status.user.screen_name
                except:
                        mined['quote_text'] = 'None'
                        mined['quote_screen_name'] = 'None'
                
                data.append(mined)
        return pd.DataFrame(data)

In [ ]:
#Mining tweets on the mavs, commented out because twitter access was removed due to too many requests for the day.

# mavs_df = mineTweets('mavs', '202202100019')
# mavs_df = pd.DataFrame()

# time = datetime.datetime.now()
# time = datetime.datetime.strptime(str(time), "%Y-%m-%d %H:%M:%S.%f").strftime('%Y%m%d%H%M')
# time = int(time)

# while time > 202202090019:
#     mavs_df = mavs_df.append(mineTweets('mavs', time))
    
#     dtTime = datetime.datetime.strptime(str(time), "%Y%m%d%H%M%S")
#     if mavs_df.tweet_id.duplicated().sum() > 0:
#         dtTime = dtTime - datetime.timedelta(minutes=5)
#     else:
#         dtTime = dtTime - datetime.timedelta(minutes=10)
#     time = int(dtTime.strftime('%Y%m%d%H%M'))

# mavs_df

NameError: name 'mineTweets' is not defined

In [3]:
#Trying SNScrape Since the twitter API rate limited the scraper

import snscrape.modules.twitter as sntwitter

def mineSntwitter(query, maxCount):
    tweet_list = []
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper(query).get_items()):
        if i>maxCount:
            break
        tweet_list.append([
                    tweet.date,
                    tweet.id,
                    tweet.user.username,
                    tweet.content,
                    tweet.user.verified,
                    tweet.retweetCount,
                    tweet.likeCount,
        ])
        tweet_df = pd.DataFrame(tweet_list, columns=['Datetime','tweet_id', 'username', 'text', 'verified', 'retweet_count', 'like_count'])

    return tweet_df

In [4]:
#Using SNScrape to scrape on tweets mentioning the mavs for the 2021-22 Season
#TODO: Scrape for a list of queries relating to the mavs/whatever team and then drop the duplicates

mavs_df = mineSntwitter('mavs since:2021-10-20 until:2022-02-10', 5000)

mavs_df.to_csv('mavs1.csv', index=False)

In [5]:
mavs_df

,Datetime,tweet_id,username,text,verified,retweet_count,like_count
0,2022-02-09 23:58:50+00:00,1491562265316773888,TheMontyShow,@603live I have been told the Mavs and Bulls a...,False,0,1
1,2022-02-09 23:58:30+00:00,1491562180872974336,neilbobnuss,@SpotOn96057010 @RGutman_ @Mavs_FFL Your first...,False,0,1
2,2022-02-09 23:58:30+00:00,1491562180281577480,tac234,"Trade for Hield, Nurkic, you know players that...",False,0,1
3,2022-02-09 23:58:25+00:00,1491562158374723585,VeritasetfidePF,"@cazefontoura Ja hj ,ninguém sabe quem é. \nSo...",False,0,4
4,2022-02-09 23:58:12+00:00,1491562107258687490,VCPHoops,"Mavs Post Game Interview: Jason Kidd, Trey Bur...",False,0,0
...,...,...,...,...,...,...,...
4996,2022-02-08 19:20:27+00:00,1491129818792214529,AtlantaVictim,Let’s just hope they win smh. The Mavs game wa...,False,0,1
4997,2022-02-08 19:20:08+00:00,1491129738953658369,MasculineFather,@All_Things_Mavs We would be Atlanta if it wer...,False,0,0
4998,2022-02-08 19:20:06+00:00,1491129733513363457,El_Rey71,@dallasmavs ur move 👀👀,False,0,0
4999,2022-02-08 19:20:05+00:00,1491129727775436801,LukaDaDon77,@All_Things_Mavs This is the time to trade Mos...,False,0,3


In [16]:
#Looping through dates to scrape tweets through a larger time period
date_df = pd.read_csv('datelist.csv')
for day in date_df['date']:
    mavs_df = pd.concat(mavs_df, mineSntwitter('mavs since:2021-10-20 until:' + day, 6000))

mavs_df.to_csv('mavs_df.csv', index=False)


KeyboardInterrupt: 

In [7]:
mavs_df

,Datetime,tweet_id,username,text,verified,retweet_count,like_count
0,2022-02-09 23:58:50+00:00,1491562265316773888,TheMontyShow,@603live I have been told the Mavs and Bulls a...,False,0,1
1,2022-02-09 23:58:30+00:00,1491562180872974336,neilbobnuss,@SpotOn96057010 @RGutman_ @Mavs_FFL Your first...,False,0,1
2,2022-02-09 23:58:30+00:00,1491562180281577480,tac234,"Trade for Hield, Nurkic, you know players that...",False,0,1
3,2022-02-09 23:58:25+00:00,1491562158374723585,VeritasetfidePF,"@cazefontoura Ja hj ,ninguém sabe quem é. \nSo...",False,0,4
4,2022-02-09 23:58:12+00:00,1491562107258687490,VCPHoops,"Mavs Post Game Interview: Jason Kidd, Trey Bur...",False,0,0
...,...,...,...,...,...,...,...
4996,2022-02-08 19:20:27+00:00,1491129818792214529,AtlantaVictim,Let’s just hope they win smh. The Mavs game wa...,False,0,1
4997,2022-02-08 19:20:08+00:00,1491129738953658369,MasculineFather,@All_Things_Mavs We would be Atlanta if it wer...,False,0,0
4998,2022-02-08 19:20:06+00:00,1491129733513363457,El_Rey71,@dallasmavs ur move 👀👀,False,0,0
4999,2022-02-08 19:20:05+00:00,1491129727775436801,LukaDaDon77,@All_Things_Mavs This is the time to trade Mos...,False,0,3
